In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import itertools
import random
from typing import List
from functools import partial
import pickle
import math
from math import comb
import re

pd.set_option("display.max_rows", 83)
pd.set_option("display.max_columns", 83)

In [2]:
from helpers import RANKS, SUITS, SMALL_STRAIGHT_RANKS, BIG_STRAIGHT_RANKS, TOTAL_CARDS_NUM, NUM_EACH_RANK, NUM_EACH_SUIT, RANK_PAIRS_DESCENDING

from Card import Card
from Deck import Deck
from counts import init_counts

In [3]:
class DeckSolver(Deck):
    def __init__(self, cards_list: List[Card] = None):
        
        super().__init__(cards_list = cards_list)

        self.ranks_counts = {rank: 0 for rank in RANKS}
        for card in self.cards:
            self.ranks_counts[card.rank] += 1

    def total_ways(self, n):
        return comb(len(self.cards), n)
    
    def ways_ranks_nums(self, n, ranks_nums):
        for rank in ranks_nums:
            if rank["rank_num"] < 0:
                raise Exception("xd")
        
        ranks_nums_sum = sum([rank["rank_num"] for rank in ranks_nums])
        if n - ranks_nums_sum < 0:
            return 0
        
        ways = 1
        
        for rank in ranks_nums:
            rank_count = self.ranks_counts[rank["rank"]]
            ways *= comb(rank_count, rank["rank_num"])
            
        rank_counts_sum = sum([self.ranks_counts[rank["rank"]] for rank in ranks_nums])
        
        return ways * comb(len(self.cards) - rank_counts_sum, n - ranks_nums_sum)

In [4]:
class Solver:
    def __init__(
        self,
        hand: List[Card],
        unknown_cards_in_play_num: int
    ):
        assert unknown_cards_in_play_num >= 0
        
        self.hand = DeckSolver(hand)
        self.deck = DeckSolver([card for card in Deck.get_all_cards() if card not in self.hand.cards])
        self.unknown_cards_in_play_num = unknown_cards_in_play_num
    
    def probability_high_card(self, rank):
        return self.probabilty_n_ranks(ranks = [
            {
                "rank": rank,
                "needed": 1,
            }
        ])
        # return self.__probabilty_n_rank(rank, 1)
    
    def probability_pair(self, rank):
        return self.probabilty_n_ranks(ranks = [
            {
                "rank": rank,
                "needed": 2,
            }
        ])
    
    def probability_three(self, rank):
        return self.probabilty_n_ranks(ranks = [
            {
                "rank": rank,
                "needed": 3,
            }
        ])
    
    def probability_quad(self, rank):
        return self.probabilty_n_ranks(ranks = [
            {
                "rank": rank,
                "needed": 4,
            }
        ])
    
    def probability_two_pair(self, rank_a, rank_b):
        return self.probabilty_n_ranks(ranks = [
            {
                "rank": rank_a,
                "needed": 2,
            },
            {
                "rank": rank_b,
                "needed": 2,
            }
        ])
    
    def probability_small_straight(self):
        return self.probabilty_n_ranks(ranks = [
            {
                "rank": "9",
                "needed": 1,
            },
            {
                "rank": "T",
                "needed": 1,
            },
            {
                "rank": "J",
                "needed": 1,
            },
            {
                "rank": "Q",
                "needed": 1,
            },
            {
                "rank": "K",
                "needed": 1,
            }
        ])
    
    def probability_big_straight(self):
        return self.probabilty_n_ranks(ranks = [
            {
                "rank": "T",
                "needed": 1,
            },
            {
                "rank": "J",
                "needed": 1,
            },
            {
                "rank": "Q",
                "needed": 1,
            },
            {
                "rank": "K",
                "needed": 1,
            },
            {
                "rank": "A",
                "needed": 1,
            }
        ])
    
    def probabilty_n_ranks(self, ranks: List[dict]):
        
        ranks = [
            {
                "rank": rank["rank"],
                "needed_actual": rank["needed"] - self.hand.ranks_counts[rank["rank"]]
            } for rank in ranks if rank["needed"] - self.hand.ranks_counts[rank["rank"]] > 0
        ]
        
        if len(ranks) == 0 or max([rank["needed_actual"] for rank in ranks]) <= 0:
            return 1
    
        n = self.unknown_cards_in_play_num
        
        if sum([rank["needed_actual"] for rank in ranks]) > n:
            return 0
        
        ranks_ranges = [
            [
                {"rank": rank["rank"], "rank_num": rank_num}
                for rank_num in range(rank["needed_actual"], NUM_EACH_RANK +  1)
            ]
            for rank in ranks
        ]
        
        positive_ranks_nums_list = itertools.product(*ranks_ranges)
    
        ways_positive = 0
        for ranks_nums in positive_ranks_nums_list:
            ways_positive += self.deck.ways_ranks_nums(n, ranks_nums)
        
        assert ways_positive > 0
        
        ways_total = self.deck.total_ways(n)
        
        return ways_positive / ways_total
        

In [5]:
hand = [
    Card("9", "♠"),
    Card("T", "♣"),
    Card("J", "♦"),
    Card("Q", "♥"),
    Card("A", "♥")
]
    
solver = Solver(
    hand = hand,
    unknown_cards_in_play_num = 1
)

solver.probabilty_n_ranks(ranks = [
    {
        "rank": "T",
        "needed": 1,
    },
    {
        "rank": "J",
        "needed": 1,
    },
    {
        "rank": "Q",
        "needed": 1,
    },
    {
        "rank": "K",
        "needed": 1,
    },
    {
        "rank": "A",
        "needed": 1,
    }
])

0.21052631578947367

In [6]:
p = init_counts()
p = p.astype(float)
    
hand = [
    Card("9", "♠"),
    Card("T", "♣"),
    Card("J", "♦"),
    Card("Q", "♥")
]

cols = []
r = list(range(2,25))
for n in r:
    
    unknown_cards_in_play_num = n - len(hand)
    
    if unknown_cards_in_play_num < 0:
        p.loc[n, :] = None
        continue
    
    solver = Solver(
        hand = hand,
        unknown_cards_in_play_num = unknown_cards_in_play_num
    )
    
    for rank in RANKS[:7]:
        # col = f'is_three_{rank}'
        # if col not in cols:
        #     cols.append(col)
        p.loc[n, f'is_high_card_{rank}'] = solver.probability_high_card(rank = rank)
        p.loc[n, f'is_pair_{rank}'] = solver.probability_pair(rank = rank)
        p.loc[n, f'is_three_{rank}'] = solver.probability_three(rank = rank)
        p.loc[n, f'is_quad_{rank}'] = solver.probability_quad(rank = rank)
        
    for (a, b) in RANK_PAIRS_DESCENDING:
        p.loc[n, f'is_two_pair_{a}_{b}'] = solver.probability_two_pair(rank_a = a, rank_b = b)
        
    p.loc[n, f'is_small_straight'] = solver.probability_small_straight()
    p.loc[n, f'is_big_straight'] = solver.probability_big_straight()


p.loc[r, cols]
p.loc[r, :]
# p[["is_small_straight", "is_big_straight"]]

,is_high_card_9,is_high_card_T,is_high_card_J,is_high_card_Q,is_high_card_K,is_high_card_A,is_pair_9,is_pair_T,is_pair_J,is_pair_Q,is_pair_K,is_pair_A,is_two_pair_T_9,is_two_pair_J_9,is_two_pair_J_T,is_two_pair_Q_9,is_two_pair_Q_T,is_two_pair_Q_J,is_two_pair_K_9,is_two_pair_K_T,is_two_pair_K_J,is_two_pair_K_Q,is_two_pair_A_9,is_two_pair_A_T,is_two_pair_A_J,is_two_pair_A_Q,is_two_pair_A_K,is_small_straight,is_big_straight,is_three_9,is_three_T,is_three_J,is_three_Q,is_three_K,is_three_A,is_full_9_T,is_full_9_J,is_full_9_Q,is_full_9_K,is_full_9_A,is_full_T_9,is_full_T_J,is_full_T_Q,is_full_T_K,is_full_T_A,is_full_J_9,is_full_J_T,is_full_J_Q,is_full_J_K,is_full_J_A,is_full_Q_9,is_full_Q_T,is_full_Q_J,is_full_Q_K,is_full_Q_A,is_full_K_9,is_full_K_T,is_full_K_J,is_full_K_Q,is_full_K_A,is_full_A_9,is_full_A_T,is_full_A_J,is_full_A_Q,is_full_A_K,is_quad_9,is_quad_T,is_quad_J,is_quad_Q,is_quad_K,is_quad_A,is_flush_♠,is_flush_♣,is_flush_♦,is_flush_♥,is_small_poker_♠,is_small_poker_♣,is_small_poker_♦,is_small_poker_♥,is_big_poker_♠,is_big_poker_♣,is_big_poker_♦,is_big_poker_♥
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,1.0,1.0,1.0,0.200000,0.200000,0.150000,0.150000,0.150000,0.150000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1.0,1.0,1.0,1.0,0.368421,0.368421,0.284211,0.284211,0.284211,0.284211,0.031579,0.031579,0.047368,0.047368,0.047368,0.047368,0.047368,0.047368,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.368421,0.084211,0.015789,0.015789,0.015789,0.015789,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1.0,1.0,1.0,1.0,0.508772,0.508772,0.403509,0.403509,0.403509,0.403509,0.087719,0.087719,0.126316,0.126316,0.126316,0.126316,0.126316,0.126316,0.015789,0.015789,0.015789,0.015789,0.015789,0.015789,0.015789,0.015789,0.000000,0.508772,0.210526,0.045614,0.045614,0.045614,0.045614,0.003509,0.003509,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000877,0.000877,0.000877,0.000877,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1.0,1.0,1.0,1.0,0.624355,0.624355,0.508772,0.508772,0.508772,0.508772,0.162023,0.162023,0.224149,0.224149,0.224149,0.224149,0.224149,0.224149,0.054489,0.054489,0.054489,0.